In [105]:
import numpy as np
import matplotlib.pyplot as plt
import control as ct
from numpy import array
import scienceplots

np.set_printoptions(precision=3, suppress=True)

plt.style.use([
    'grid',
    'retro'
])

In [106]:
Ms = 1000
Mu = 100

Ks = 30000
Ku = 30000

C = 500

M = np.diag([Ms, Mu])

K = np.array([
    [Ks, -Ks],
    [-Ks, Ks+Ku]
])

C = np.array([
    [C, -C],
    [-C, C]
])

N = M.shape[0]

In [107]:
_M = -np.linalg.inv(M)

A = np.block([
    [np.zeros((N, N)), np.eye(N)],
    [_M@K, _M@C]
])

F = np.array([
    [0,  0],
    [Ks, Ku],
])

B = np.block([
    [np.zeros((N, N))],
    [-_M@F]
])

_C = np.eye(2*N)
D = np.zeros((2*N, N))

sys = ct.ss(A, B, _C, D)
tf_sys = ct.ss2tf(sys)

In [108]:
B

array([[  0.,   0.],
       [  0.,   0.],
       [  0.,   0.],
       [300., 300.]])

In [109]:
%matplotlib qt

ct.bode(tf_sys[0,0], omega_limits=(1e-1, 1e3))
ct.bode(tf_sys[2,0], omega_limits=(1e-1, 1e3))

plt.legend(['$x_u$', '$x_s$'])

In [111]:
T = np.linspace(0, 100, 1000)
U = np.zeros((2, T.shape[0]))

U[0, :] = np.sin(2*np.pi*8*T)

_,y = ct.forced_response(
    sys, T, U
)

plt.plot(T, y[0])
plt.plot(T, y[1])